In [1]:
'''
  Install packages required to connect to Clooud MySQL instance
'''
# Install PyMySQL python module
!pip install cloud-sql-python-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.30.0 which is incompatible.


In [1]:
# Install PyMySQL python module
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.3 MB/s eta 0:00:00


In [2]:
'''
  Configure Google Cloud Project Context
'''
from tools import load_db_cfg
dbcfg= load_db_cfg('cfg.json')
!gcloud config set project {dbcfg['project_id']}

Updated property [core/project].


In [3]:
'''
  Aunthenticate to Google Colab
'''
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install -q streamlit
!pip install streamlit-option-menu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 9.8 MB/s eta 0:00:00


In [12]:
%%writefile youtube_stats.py

import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import numpy as np

from pathlib import Path
import sys
import traceback

sys.path.append(str(Path('ytdao.py').resolve()))
from ytdao import YtDao
from ytapi import get_channel_info,get_playlist_info,get_comments_info,get_video_info
from tools import load_db_cfg, load_api_cfg

# Load the DB config and initialize the DAO
dao = YtDao(load_db_cfg('cfg.json'))

# Get the channel and stats info from DB
dfsc= dao.get_supported_channels()
dsss = dao.get_supported_stats()

#Load the API config and initialize API service endpoint
import googleapiclient.discovery
apicfg= load_api_cfg('cfg.json')
youtube = googleapiclient.discovery.build(apicfg['service_name'], apicfg['version'], developerKey=apicfg['key'])

st.set_page_config(layout='wide')

# Initialize session state variable, which are to be used later

if 'channel' not in st.session_state:
  st.session_state.channel = None

if 'playlists' not in st.session_state:
  st.session_state.playlists = None

if  'comments' not in st.session_state:
  st.session_state.comments = None

if 'videos' not in st.session_state:
  st.session_state.videos = None


# Initialize the required session state variables
if 'channel_sb' not in st.session_state:
  st.session_state.channel_sb = None

if 'channel_ti' not in st.session_state:
  st.session_state.channel_ti = None

if 'save_button' not in st.session_state:
  st.session_state.save_button= None

if 'view_button' not in st.session_state:
  st.session_state.view_button= None

if 'stat_sb' not in st.session_state:
  st.session_state.stat_sb= None

# Divide the screen into two parts - Left Navigation Menu and Main View
menu,main = st.columns([10,90])

with st.sidebar:
  menu_selection=option_menu( menu_title='Actions', options=['Summary', 'Collect', 'Analyze'])

# Display the title and insert a divder under it
main.header('YouTube Channel Statistics')
main.divider()

# Handler that is called when a channel is selected from the dropdown
def on_channel_select():
  if st.session_state.channel_sb is not None:
    channel_id = dfsc[dfsc['name'] == st.session_state.channel_sb].reset_index().iloc[0]['id']
    df=dao.get_channel(channel_id)
    #data_view.dataframe(df.T,key='dfsummary',use_container_width=True)
    data_view.table(df.T)

# Handler that is called when a channel ID is typed into the text input and 'Enter' key is pressed
def on_text_input_change():
  pass


def on_stat_select():
    df=dao.get_stat(dsss[dsss==st.session_state.stat_sb].index[0])
    #data_view.dataframe(df,key='dfstat',use_container_width=True)
    data_view.table(df)


def on_view_button_click():
  try:
    data_view.text('''This operation may take a few mins... Please standby...''')
    # Get channel info from Youtybe Data API v3
    channel = get_channel_info(st.session_state.channel_ti)
    playlists = get_playlist_info(st.session_state.channel_ti)
    comments = get_comments_info(st.session_state.channel_ti)
    videos =get_video_info(channel)
  except:
    data_view.text('''Failed to fetch information using YouTube Data v3 APIs''')
    return

  try:
    data_view.text('\nChannel Details:')
    data_view.dataframe(channel,key='dfchannel_details',use_container_width=True)
    st.session_state.channel = channel

    data_view.text('\nPlaylist Details:')
    playlist_df = pd.DataFrame(playlists).T
    data_view.dataframe(playlist_df,key='dfplaylist_details',use_container_width=True)
    st.session_state.playlists = playlists

    data_view.text('\nComment Details:')
    comments_df = pd.DataFrame(comments).T
    data_view.dataframe(comments_df,key='dfcomments_details',use_container_width=True)
    st.session_state.comments = comments

    data_view.text('\nVideo Details:')
    videos_df = pd.DataFrame(videos).T
    data_view.dataframe(videos_df, key='dfvideos_details',use_container_width=True)
    st.session_state.videos = videos
  except:
      data_view.markdown(f''':red[Error while displaying results !!!]''')

def on_save_button_click():
  if dfsc[dfsc['id']==st.session_state.channel_ti].size ==0:
    if st.session_state.channel is not None and st.session_state.playlists is not None and st.session_state.comments is not None and  st.session_state.videos:
      dao.add_channel_info(st.session_state.channel, st.session_state.playlists, st.session_state.comments, st.session_state.videos)
      data_view.text(f'''Successfully Saved Channel ID: {st.session_state.channel_ti}''')
    else:
      data_view.markdown(f''':red[Error while writing to database !!!]''')
  else:
    data_view.markdown(f''':red[Channel already exists in the database !!! Please try another one !!]''')
    #data_view.text(f'''Channel with Id {st.session_state.channel_ti} already exist in the database.''')

if menu_selection == 'Summary':
  # Render the channel selector dropdown
  try:
    main.subheader('View summary of a saved channel')
    st.session_state.channel_sb = main.selectbox("Select a Channel", options=dfsc.name, key='channel_sb',placeholder='Select a Channel...', on_change=on_channel_select)
  except:
    print('')
elif menu_selection=='Collect':
  # Render the channel ID text input
  try:
    main.subheader('''Save a channel after reviewing the channel details''')
    text_container, view_button_container, save_button_container = main.columns([84,8,8])
    st.session_state.channel_ti=text_container.text_input('Enter a new channel:', key='channel_ti', on_change=on_text_input_change)
  except:
    print('')

  # Render the Channel Info View button
  try:
    view_button_container.write('')
    view_button_container.write('')
    st.session_state.view_button=view_button_container.button('Review', key='view_button', on_click=on_view_button_click)
  except:
    print('')

  # Render the Channel Info Save button
  try:
      save_button_container.write('')
      save_button_container.write('')
      st.session_state.save_button=save_button_container.button('Save', key='save_button', on_click=on_save_button_click)
  except:
    print('')
elif menu_selection=='Analyze':
  # Render the statistics selector dropdown
  try:
    main.subheader('''Insights from saved channels''')
    st.session_state.stat_sb=main.selectbox("Select a Statistic", options=dsss, key='stat_sb',placeholder='Select a Stat...', on_change=on_stat_select)
  except:
    print('')

# Create containers to place the results
data_view= main.container()


Overwriting youtube_stats.py


In [6]:
! wget -q -O - ipv4.icanhazip.com

34.106.112.234


In [ ]:
!streamlit run /content/youtube_stats.py &>/content/logs.txt &
!npx localtunnel --port 8501




npx: installed 22 in 2.359s
your url is: https://quick-cups-repeat.loca.lt
